In [1]:
import semantic_kernel as sk

kernel = sk.Kernel()

In [2]:
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

api_key, org_id = sk.openai_settings_from_dot_env()

gpt_model = "gpt-3.5-turbo-1106"
# gpt_model = "gpt-3.5-turbo"

kernel.add_chat_service("chat-gpt", OpenAIChatCompletion(ai_model_id=gpt_model, api_key=api_key, org_id=org_id))

In [3]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas. She speaks Chinese so write it in Chinese.
"""

In [4]:
from semantic_kernel.core_skills.text_skill import TextSkill

skills_directory = "../samples/skills/"
summarize_skill = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")
writer_skill = kernel.import_semantic_skill_from_directory(skills_directory, "WriterSkill")
text_skill = kernel.import_skill(TextSkill(), "TextSkill")

In [5]:
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

In [6]:
basic_plan = await planner.create_plan_async(ask, kernel)

In [7]:
print(basic_plan.generated_plan)

{
    "input": "Valentine's Day Date Ideas",
    "subtasks": [
        {"function": "WriterSkill.Brainstorm"},
        {"function": "WriterSkill.Translate", "args": {"language": "Chinese"}}
    ]
}


In [9]:
results = await planner.execute_plan_async(basic_plan, kernel)

In [10]:
print(results)

1. 在一家高档餐厅享受浪漫的烛光晚餐
2. 在公园野餐，吃着心形三明治和起泡酒
3. 一起做一顿自制晚餐
4. 去风景优美的远足，顶部享受野餐
5. 一起去情侣水疗中心，享受按摩和面部护理
6. 去山上一个舒适的小木屋度过浪漫的周末
7. 在家里看你最喜欢的浪漫电影
8. 乘船观赏日落，享受香槟祝酒
9. 一起参加舞蹈课程，学习一种新的舞蹈风格
10. 参加当地葡萄园的品酒之旅


## Action Planner

In [11]:
from semantic_kernel.planning import ActionPlanner
planner = ActionPlanner(kernel)

In [12]:
from semantic_kernel.core_skills import FileIOSkill, MathSkill, TextSkill, TimeSkill
kernel.import_skill(MathSkill(), "math")
kernel.import_skill(FileIOSkill(), "fileIO")
kernel.import_skill(TimeSkill(), "time")
kernel.import_skill(TextSkill(), "text")

{'lowercase': SKFunction(FUNCTION_PARAM_NAME_REGEX='^[0-9A-Za-z_]*$', FUNCTION_NAME_REGEX='^[0-9A-Za-z_]*$', SKILL_NAME_REGEX='^[0-9A-Za-z_]*$'),
 'trim': SKFunction(FUNCTION_PARAM_NAME_REGEX='^[0-9A-Za-z_]*$', FUNCTION_NAME_REGEX='^[0-9A-Za-z_]*$', SKILL_NAME_REGEX='^[0-9A-Za-z_]*$'),
 'trim_end': SKFunction(FUNCTION_PARAM_NAME_REGEX='^[0-9A-Za-z_]*$', FUNCTION_NAME_REGEX='^[0-9A-Za-z_]*$', SKILL_NAME_REGEX='^[0-9A-Za-z_]*$'),
 'trim_start': SKFunction(FUNCTION_PARAM_NAME_REGEX='^[0-9A-Za-z_]*$', FUNCTION_NAME_REGEX='^[0-9A-Za-z_]*$', SKILL_NAME_REGEX='^[0-9A-Za-z_]*$'),
 'uppercase': SKFunction(FUNCTION_PARAM_NAME_REGEX='^[0-9A-Za-z_]*$', FUNCTION_NAME_REGEX='^[0-9A-Za-z_]*$', SKILL_NAME_REGEX='^[0-9A-Za-z_]*$')}

In [13]:
ask = "What is the sum of 110 and 990?"

In [14]:
plan = await planner.create_plan_async(goal=ask)

In [15]:
result = await plan.invoke_async()

In [16]:
print(result)

1100


## Stepwise Planner

In [7]:
from semantic_kernel.planning import StepwisePlanner
from semantic_kernel.planning.stepwise_planner.stepwise_planner_config import (
    StepwisePlannerConfig,
)

In [8]:
class WebSearchEngineSkill:
    """
    A search engine skill.
    """
    from semantic_kernel.orchestration.sk_context import SKContext
    from semantic_kernel.skill_definition import sk_function, sk_function_context_parameter

    def __init__(self, connector) -> None:
        self._connector = connector

    @sk_function(
        description="Performs a web search for a given query", name="searchAsync"
    )
    @sk_function_context_parameter(
        name="query",
        description="The search query",
    )
    async def search_async(self, query: str, context: SKContext) -> str:
        query = query or context.variables.get("query")[1]
        result = await self._connector.search_async(query, num_results=5, offset=0)
        return str(result)

In [9]:
from semantic_kernel.connectors.search_engine import BingConnector

BING_API_KEY = sk.bing_search_settings_from_dot_env()
connector = BingConnector(BING_API_KEY)
kernel.import_skill(WebSearchEngineSkill(connector), skill_name="WebSearch")

{'searchAsync': SKFunction(FUNCTION_PARAM_NAME_REGEX='^[0-9A-Za-z_]*$', FUNCTION_NAME_REGEX='^[0-9A-Za-z_]*$', SKILL_NAME_REGEX='^[0-9A-Za-z_]*$')}

In [10]:
from semantic_kernel.core_skills.math_skill import MathSkill
from semantic_kernel.core_skills.time_skill import TimeSkill

kernel.import_skill(TimeSkill(), "time")
kernel.import_skill(MathSkill(), "math")

{'Add': SKFunction(FUNCTION_PARAM_NAME_REGEX='^[0-9A-Za-z_]*$', FUNCTION_NAME_REGEX='^[0-9A-Za-z_]*$', SKILL_NAME_REGEX='^[0-9A-Za-z_]*$'),
 'Subtract': SKFunction(FUNCTION_PARAM_NAME_REGEX='^[0-9A-Za-z_]*$', FUNCTION_NAME_REGEX='^[0-9A-Za-z_]*$', SKILL_NAME_REGEX='^[0-9A-Za-z_]*$')}

In [11]:
planner = StepwisePlanner(
    kernel, StepwisePlannerConfig(max_iterations=10, min_iteration_time_ms=1000)
)

In [12]:
ask = """Which NBA team has won the most championships?"""

plan = planner.create_plan(goal=ask)

In [13]:
result = await plan.invoke_async()

In [14]:
print(result)

The search did not return the expected results. I will attempt to use a different approach to find the answer to the question.


In [15]:
for index, step in enumerate(plan._steps):
    print("Step:", index)
    print("Description:",step.description)
    print("Function:", step.skill_name + "." + step._function.name)
    if len(step._outputs) > 0:
        print( "  Output:\n", str.replace(result[step._outputs[0]],"\n", "\n  "))

Step: 0
Description: Execute a plan
Function: StepwisePlanner.ExecutePlan
  Output:
 This was my previous work (but they haven't seen any of it! They only see what I return as final answer):
  [THOUGHT]
  I can use the WebSearch.searchAsync function to perform a web search for the query "NBA team with most championships" and find the answer.
  [ACTION]
  {"action": "WebSearch.searchAsync", "action_variables": {"query": "NBA team with most championships"}}
  [OBSERVATION]
  ["Bitcoin prices surge for first time since April Cointelegraph Spot Bitcoin ETFs could be rejected if the SEC wants 'more time' — Analyst 1 hour ago Bloomberg.com", '#1 Bitcoin BTCUSDBitstamp BTCUSDBitstamp 44293USD +1428 +3.33% As of today at 14:48 UTC-8 See on Supercharts Overview Timeline News Ideas Technicals Markets BTCUSD chart Price Market Cap Today −4.79% Week 0.71% 1 month 7.06% 6 months 43.18% Year to date 1.31% 1 year 156.86% 5 years 1030.93% All time 392679.82% Key stats Market capitalization', 'All Abou